In [40]:
import numpy as np
from numpy import sqrt, log
import random
import matplotlib.pyplot as plt
import os
import math
from numpy import exp
from scipy.sparse import diags

In [2]:
directory='/Users/julnow/Desktop/szkoła/KMS/2/'
params = directory + 'params.txt'


    Przesyłam zadanie drugie - jest to ex. 19 z przesłąnego materiału. Wcześniej należy przeczytać podp. 5.2.4.
Celem będzie rozwiazanie jednowymairowego równania Schrodingera zaleznego od czasu, korzystając z operatora ewolucji czasowej w postaci Cayleya (wz. 5.86) - było o tym trochę na ostatnim wykładzie przy okazji metody TDHF. Startujemy z funkcji (5.91) a nastepnie propagujemy taki impuls falowy uwzględniając próg potencjału z podpunktu 19b. Resztę doprecyzuję na zajęciach, ale procedurę budujaca macierz (lewa strona wz. 5.89) można sobie już przygotować. Do rozwiązania numerycznego układu równań (5.89) stosujemy oczywiscie funkcje biblioteczne.


In [60]:
#imaginary unit in numpy = 1j
pi = np.pi

#stale
x0 = -5
x1 = 25
x_steps = 200
x = np.linspace(x0, x1, x_steps)
dx = abs(x0-x1)/x_steps
v0 = 105
sigma = 0.5
k0 = 10
dk = 1
t0 = 0
t1 = 1
dt = 0.005
#phi0
phi = sqrt(dk) / pow(pi, 1/4) * exp(- (x**2 * dk**2 / 2)) * exp(1j* k0 * x)
#v0
v = v0 * exp (- ( x-10)**2 / sigma**2)
# 5.90
b =  1 + 1j * dt  / 2 * ( 2/dx**2 + v )
a = c = np.full( (len(b) - 1), - 1j*dt/2/ dx**2)
#diagonal matrix using scipy function
offset = [-1,0,1]
A = diags([a, b, c] ,offset).toarray()